# Assignment 3


- Student Name: Karina Jonina
- Student ID: c00278440

- Module Name: Programming for Data Scientists
- Module Code: PROGC5201
- Module Leader: Paul Barry

- Course Name: 		Master of Science in Data Science
- Course Code:		 CW_KCDAR_M Y5

- Due Date: 10th January 2022 9:00am

In [1]:
import pandas as pd
import datetime
from os import listdir
from os.path import isfile, join
import glob
import re
from datetime import datetime

In [2]:
# gettting the CSV from the website
url = 'https://opendata-geohive.hub.arcgis.com/datasets/d8eb52d56273413b84b0187a4e9117be_0.csv?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D.'
# url = 'COVID-19.csv'
df = pd.read_csv(url)


In [3]:
# Ensuring the Date is just date, and not datetime
df['Date'] = pd.to_datetime(df['Date']).dt.date
df.head()

,X,Y,Date,ConfirmedCovidCases,TotalConfirmedCovidCases,ConfirmedCovidDeaths,TotalCovidDeaths,StatisticsProfileDate,CovidCasesConfirmed,HospitalisedCovidCases,...,CommunityTransmission,CloseContact,TravelAbroad,FID,HospitalisedAged65to74,HospitalisedAged75to84,HospitalisedAged85up,Aged65to74,Aged75to84,Aged85up
0,-7.692596,53.288234,2020-02-29,1,1,0.0,0,2020/02/27 00:00:00+00,NaN,NaN,...,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN
1,-7.692596,53.288234,2020-03-03,1,2,0.0,0,2020/03/01 00:00:00+00,NaN,NaN,...,0,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN
2,-7.692596,53.288234,2020-03-04,4,6,0.0,0,2020/03/02 00:00:00+00,1.0,0.0,...,0,0,0,3,0.0,0.0,0.0,0.0,0.0,0.0
3,-7.692596,53.288234,2020-03-05,7,13,0.0,0,2020/03/03 00:00:00+00,2.0,1.0,...,0,0,0,4,0.0,0.0,0.0,0.0,0.0,0.0
4,-7.692596,53.288234,2020-03-06,5,18,0.0,0,2020/03/04 00:00:00+00,5.0,4.0,...,0,0,0,5,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# examining columns and rows
df.shape

(647, 41)

In [5]:
# checking type for each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 647 entries, 0 to 646
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   X                            647 non-null    float64
 1   Y                            647 non-null    float64
 2   Date                         647 non-null    object 
 3   ConfirmedCovidCases          647 non-null    int64  
 4   TotalConfirmedCovidCases     647 non-null    int64  
 5   ConfirmedCovidDeaths         499 non-null    float64
 6   TotalCovidDeaths             647 non-null    int64  
 7   StatisticsProfileDate        647 non-null    object 
 8   CovidCasesConfirmed          645 non-null    float64
 9   HospitalisedCovidCases       645 non-null    float64
 10  RequiringICUCovidCases       645 non-null    float64
 11  HealthcareWorkersCovidCases  645 non-null    float64
 12  ClustersNotified             625 non-null    float64
 13  HospitalisedAged5   

In [6]:
# examining the names of the columns
df.columns

Index(['X', 'Y', 'Date', 'ConfirmedCovidCases', 'TotalConfirmedCovidCases',
       'ConfirmedCovidDeaths', 'TotalCovidDeaths', 'StatisticsProfileDate',
       'CovidCasesConfirmed', 'HospitalisedCovidCases',
       'RequiringICUCovidCases', 'HealthcareWorkersCovidCases',
       'ClustersNotified', 'HospitalisedAged5', 'HospitalisedAged5to14',
       'HospitalisedAged15to24', 'HospitalisedAged25to34',
       'HospitalisedAged35to44', 'HospitalisedAged45to54',
       'HospitalisedAged55to64', 'Male', 'Female', 'Unknown', 'Aged1to4',
       'Aged5to14', 'Aged15to24', 'Aged25to34', 'Aged35to44', 'Aged45to54',
       'Aged55to64', 'Median_Age', 'CommunityTransmission', 'CloseContact',
       'TravelAbroad', 'FID', 'HospitalisedAged65to74',
       'HospitalisedAged75to84', 'HospitalisedAged85up', 'Aged65to74',
       'Aged75to84', 'Aged85up'],
      dtype='object')

## Examining location
Below exploration showed that there are no unique locations in the dataset and therefore, redundant. 

In [7]:
df['X'].unique()

array([-7.692596])

In [8]:
df['Y'].unique()

array([53.288234])

In [9]:
df['CommunityTransmission'].unique()

array([0], dtype=int64)

In [10]:
df['CloseContact'].unique()

array([0], dtype=int64)

In [11]:
df['TravelAbroad'].unique()

array([0], dtype=int64)

### Dropping Columns (X and Y)

| Column | Reason |
|---|---|
| df['X'] | no unique values |
| df['Y'] | no unique values |
| df['StatisticsProfileDate'] | Very similar to df['Date']|
| df['CovidCasesConfirmed'] | Very similar to df['Total Confirmed Covid Cases'] |
| df['FID'] | very similar to Index in pandas |
| df['CommunityTransmission'] | no unique values |
| df['CloseContact'] | no unique values |
| df['TravelAbroad'] | no unique values |


In [12]:
# Dropping irrelevant columns
df = df.drop(columns = ['X','Y','CovidCasesConfirmed','StatisticsProfileDate', 'FID', 
                        'CommunityTransmission', 'CloseContact','TravelAbroad'])

## Examining date
Below exploration showed that there are are unique dates in the dataset. 

In [13]:
df['Date'].unique()

array([datetime.date(2020, 2, 29), datetime.date(2020, 3, 3),
       datetime.date(2020, 3, 4), datetime.date(2020, 3, 5),
       datetime.date(2020, 3, 6), datetime.date(2020, 3, 7),
       datetime.date(2020, 3, 8), datetime.date(2020, 3, 9),
       datetime.date(2020, 3, 10), datetime.date(2020, 3, 11),
       datetime.date(2020, 3, 12), datetime.date(2020, 3, 13),
       datetime.date(2020, 3, 14), datetime.date(2020, 3, 15),
       datetime.date(2020, 3, 16), datetime.date(2020, 3, 17),
       datetime.date(2020, 3, 18), datetime.date(2020, 3, 19),
       datetime.date(2020, 3, 20), datetime.date(2020, 3, 21),
       datetime.date(2020, 3, 22), datetime.date(2020, 3, 23),
       datetime.date(2020, 3, 24), datetime.date(2020, 3, 25),
       datetime.date(2020, 3, 26), datetime.date(2020, 3, 27),
       datetime.date(2020, 3, 28), datetime.date(2020, 3, 29),
       datetime.date(2020, 3, 30), datetime.date(2020, 3, 31),
       datetime.date(2020, 4, 1), datetime.date(2020, 4, 2),
 

## Melting 

In [14]:
# Melting
id_vars = ["Date"]

df = pd.melt(frame=df,id_vars=id_vars, var_name="Column", value_name="Cases")

df.head(10)


,Date,Column,Cases
0,2020-02-29,ConfirmedCovidCases,1.0
1,2020-03-03,ConfirmedCovidCases,1.0
2,2020-03-04,ConfirmedCovidCases,4.0
3,2020-03-05,ConfirmedCovidCases,7.0
4,2020-03-06,ConfirmedCovidCases,5.0
5,2020-03-07,ConfirmedCovidCases,1.0
6,2020-03-08,ConfirmedCovidCases,2.0
7,2020-03-09,ConfirmedCovidCases,3.0
8,2020-03-10,ConfirmedCovidCases,10.0
9,2020-03-11,ConfirmedCovidCases,9.0


# Getting the final dataset

In [15]:
# # Trying to get OR working
# tmp_df = df["Column"].str.extract("(\D+)(\d+)(\D{2})(\d{2}) | (\D+)(\d{2}) | (\D+)(\d+)(\D{2})", expand=False)    

# print(tmp_df)

In [16]:
tmp_df = df["Column"].str.extract("(\D+)(\d+)(\D{2})(\d{2})", expand=False)    

# Name columns
tmp_df.columns = ["Aged", "age_lower","to", "age_upper"]

# Create `age`column based on `age_lower` and `age_upper`
tmp_df["Age_Range"] = tmp_df["age_lower"] + "-" + tmp_df["age_upper"]

tmp_df.head(15)

,Aged,age_lower,to,age_upper,Age_Range
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN


In [17]:
# Merge 
df = pd.concat([df, tmp_df], axis=1)

# Drop unnecessary columns and rows
df = df.drop(["age_lower","to","age_upper" ], axis=1)

# sorting data by date
df = df.sort_values(ascending = True, by = ["Date"])

# reseting the dataframe index
df = df.reset_index()


In [18]:
df["Column"].unique()

array(['ConfirmedCovidCases', 'HospitalisedAged85up',
       'HospitalisedAged75to84', 'TotalConfirmedCovidCases',
       'HospitalisedAged65to74', 'Median_Age', 'Aged55to64', 'Aged45to54',
       'ConfirmedCovidDeaths', 'Aged35to44', 'Aged25to34', 'Aged15to24',
       'Aged5to14', 'Aged85up', 'Aged65to74', 'Aged1to4', 'Unknown',
       'Female', 'Male', 'HospitalisedCovidCases',
       'HospitalisedAged55to64', 'HospitalisedAged45to54',
       'HospitalisedAged35to44', 'HospitalisedAged25to34',
       'HospitalisedAged15to24', 'RequiringICUCovidCases',
       'HospitalisedAged5to14', 'HospitalisedAged5', 'ClustersNotified',
       'TotalCovidDeaths', 'Aged75to84', 'HealthcareWorkersCovidCases'],
      dtype=object)

In [19]:
df = df [[
    "Date",
    "Column",
    "Age_Range",
    "Cases"
]]

In [20]:
df.head()

,Date,Column,Age_Range,Cases
0,2020-02-29,ConfirmedCovidCases,NaN,1.0
1,2020-02-29,HospitalisedAged85up,NaN,NaN
2,2020-02-29,HospitalisedAged75to84,75-84,NaN
3,2020-02-29,TotalConfirmedCovidCases,NaN,1.0
4,2020-02-29,HospitalisedAged65to74,65-74,NaN


In [21]:
# checking that 'HospitalisedAged5' with have 5
df[df["Column"] =='HospitalisedAged5'].head()

,Date,Column,Age_Range,Cases
27,2020-02-29,HospitalisedAged5,NaN,NaN
45,2020-03-03,HospitalisedAged5,NaN,NaN
91,2020-03-04,HospitalisedAged5,NaN,0.0
111,2020-03-05,HospitalisedAged5,NaN,0.0
156,2020-03-06,HospitalisedAged5,NaN,0.0


In [22]:
# checking that 'HospitalisedAged85up' with have 85
df[df["Column"] =='HospitalisedAged85up'].head()

,Date,Column,Age_Range,Cases
1,2020-02-29,HospitalisedAged85up,NaN,NaN
55,2020-03-03,HospitalisedAged85up,NaN,NaN
78,2020-03-04,HospitalisedAged85up,NaN,0.0
106,2020-03-05,HospitalisedAged85up,NaN,0.0
136,2020-03-06,HospitalisedAged85up,NaN,0.0


### Dropping rows with NaN in cases

In [23]:
df1 = df.copy()

In [24]:
df1.head(20)

,Date,Column,Age_Range,Cases
0,2020-02-29,ConfirmedCovidCases,NaN,1.0
1,2020-02-29,HospitalisedAged85up,NaN,NaN
2,2020-02-29,HospitalisedAged75to84,75-84,NaN
3,2020-02-29,TotalConfirmedCovidCases,NaN,1.0
4,2020-02-29,HospitalisedAged65to74,65-74,NaN
5,2020-02-29,Median_Age,NaN,NaN
6,2020-02-29,Aged55to64,55-64,NaN
7,2020-02-29,Aged45to54,45-54,NaN
8,2020-02-29,ConfirmedCovidDeaths,NaN,0.0
9,2020-02-29,Aged35to44,35-44,NaN


In [25]:
df1 = df1[~df1['Cases'].isna()]
df1["Cases"] = df1["Cases"].astype(int)
# reseting the dataframe index
df1 = df1.reset_index(drop = True)

In [26]:
df1.head()

,Date,Column,Age_Range,Cases
0,2020-02-29,ConfirmedCovidCases,NaN,1
1,2020-02-29,TotalConfirmedCovidCases,NaN,1
2,2020-02-29,ConfirmedCovidDeaths,NaN,0
3,2020-02-29,TotalCovidDeaths,NaN,0
4,2020-03-03,TotalCovidDeaths,NaN,0


In [27]:
df1.shape

(20480, 4)

### Dealing with ['Column'] column

In [28]:
# Testing the regular expression
line = 'HospitalisedAged45to54'
match = re.findall('([A-Z][a-z]+)', line)
new_line = match[0] + " " + match[1] 
print(new_line)

Hospitalised Aged


In [29]:
column_list = []
for i in  df1["Column"]:
    match = re.findall('([A-Z][a-z]+)', i)
#     print(match)
    if len(match) == 4:
        new_line = match[0] + ' ' + match[1] + ' ' + match[2] + ' ' + match[3]
    if len(match) == 3:
        new_line = match[0] + ' ' + match[1] + ' ' + match[2]
    elif len(match) == 2:
        new_line = match[0] + ' ' + match[1] 
#     elif len(match) == 1:
#         new_line = match[0]
    else:
        new_line = match[0]
        print(new_line)
    
    column_list.append(new_line)

# examining the list
# print(column_list)  

Total
Total
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Female
Unknown
Aged
Total
Male
Aged
Aged
Healthcare
Unknown
Aged
Aged
Total
Aged
Aged
Aged
Aged
Aged
Healthcare
Aged
Aged
Female
Aged
Male
Female
Male
Unknown
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Total
Aged
Aged
Healthcare
Total
Aged
Aged
Aged
Aged
Aged
Aged
Male
Healthcare
Female
Aged
Aged
Aged
Aged
Unknown
Aged
Aged
Aged
Aged
Aged
Female
Aged
Healthcare
Unknown
Aged
Aged
Male
Aged
Aged
Total
Female
Aged
Total
Male
Aged
Unknown
Healthcare
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Unknown
Healthcare
Aged
Total
Aged
Male
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Female
Female
Aged
Total
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Unknown
Aged
Aged
Healthcare
Male
Aged
Aged
Healthcare
Aged
Aged
Aged
Total
Female
Aged
Aged
Aged
Male
Aged
Unknown
Aged
Aged
Aged
Aged
Healthcare
Aged
Aged
Aged
Aged
Male
Unknown
Total
Aged
Aged
Female
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Healthcare
Unknown
Male
Female
Total
Aged
Unknown
Aged
Male
Healthcare
Ag

Aged
Aged
Aged
Total
Unknown
Aged
Healthcare
Aged
Aged
Female
Male
Aged
Aged
Aged
Aged
Female
Aged
Healthcare
Aged
Unknown
Aged
Aged
Aged
Male
Aged
Aged
Total
Aged
Total
Aged
Aged
Aged
Healthcare
Aged
Unknown
Aged
Aged
Aged
Female
Male
Aged
Aged
Female
Aged
Aged
Aged
Healthcare
Aged
Aged
Aged
Aged
Male
Aged
Aged
Total
Unknown
Aged
Aged
Unknown
Aged
Aged
Aged
Aged
Aged
Male
Aged
Aged
Total
Aged
Healthcare
Aged
Female
Male
Aged
Aged
Aged
Aged
Healthcare
Female
Aged
Aged
Aged
Aged
Unknown
Total
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Unknown
Aged
Aged
Aged
Healthcare
Female
Total
Male
Aged
Aged
Aged
Aged
Unknown
Aged
Aged
Female
Aged
Healthcare
Aged
Total
Aged
Aged
Aged
Aged
Male
Female
Unknown
Aged
Total
Aged
Aged
Male
Aged
Healthcare
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Female
Aged
Aged
Healthcare
Aged
Total
Aged
Aged
Male
Unknown
Aged
Aged
Healthcare
Female
Aged
Aged
Aged
Aged
Aged
Male
Unknown
Aged
Aged
Aged
Total
Aged
Aged
Aged
Healthcare
Unknown
Aged
Total
Aged
Aged
Male
Age

Male
Aged
Aged
Aged
Female
Aged
Aged
Aged
Aged
Healthcare
Aged
Aged
Unknown
Total
Aged
Aged
Female
Unknown
Aged
Healthcare
Aged
Aged
Aged
Aged
Total
Male
Aged
Aged
Aged
Aged
Aged
Aged
Male
Aged
Healthcare
Aged
Aged
Aged
Aged
Aged
Aged
Female
Total
Unknown
Aged
Aged
Aged
Aged
Aged
Male
Healthcare
Unknown
Female
Aged
Aged
Aged
Total
Aged
Aged
Aged
Aged
Total
Unknown
Aged
Aged
Healthcare
Aged
Aged
Aged
Female
Aged
Male
Aged
Aged
Aged
Aged
Total
Unknown
Aged
Aged
Aged
Aged
Aged
Aged
Female
Aged
Healthcare
Aged
Male
Healthcare
Total
Aged
Female
Male
Aged
Aged
Aged
Aged
Aged
Aged
Unknown
Aged
Aged
Aged
Aged
Healthcare
Aged
Male
Female
Aged
Aged
Aged
Unknown
Total
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Male
Aged
Aged
Aged
Healthcare
Unknown
Aged
Female
Total
Aged
Aged
Female
Aged
Aged
Aged
Aged
Aged
Total
Aged
Unknown
Aged
Aged
Aged
Male
Aged
Healthcare
Unknown
Aged
Total
Aged
Aged
Aged
Aged
Aged
Aged
Healthcare
Aged
Female
Aged
Male
Aged
Aged
Aged
Male
Aged
Aged
Female
Aged
Aged
Total


Total
Aged
Aged
Unknown
Aged
Aged
Healthcare
Aged
Aged
Aged
Aged
Aged
Male
Aged
Female
Aged
Aged
Aged
Aged
Unknown
Aged
Healthcare
Female
Aged
Aged
Total
Aged
Aged
Aged
Male
Aged
Male
Unknown
Aged
Aged
Aged
Aged
Total
Aged
Aged
Aged
Healthcare
Aged
Female
Aged
Aged
Aged
Aged
Female
Male
Aged
Total
Aged
Aged
Healthcare
Unknown
Aged
Aged
Aged
Aged
Aged
Aged
Female
Total
Male
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Healthcare
Unknown
Aged
Aged
Total
Aged
Aged
Male
Unknown
Aged
Aged
Aged
Healthcare
Female
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Female
Unknown
Total
Aged
Aged
Aged
Aged
Aged
Male
Healthcare
Aged
Aged
Total
Healthcare
Aged
Female
Aged
Aged
Aged
Male
Aged
Aged
Aged
Aged
Unknown
Unknown
Aged
Aged
Healthcare
Aged
Aged
Male
Aged
Aged
Aged
Total
Female
Aged
Aged
Aged
Aged
Female
Aged
Total
Aged
Aged
Healthcare
Aged
Aged
Unknown
Aged
Aged
Aged
Aged
Male
Aged
Total
Aged
Aged
Male
Aged
Aged
Female
Aged
Aged
Aged
Unknown
Aged
Healthcare
Aged
Aged
Healthcare
Aged
Total
Aged
Aged
Aged
A

Aged
Aged
Aged
Aged
Aged
Total
Aged
Aged
Aged
Aged
Female
Male
Aged
Aged
Healthcare
Aged
Unknown
Unknown
Aged
Male
Aged
Aged
Healthcare
Aged
Aged
Aged
Aged
Aged
Total
Female
Aged
Aged
Unknown
Aged
Aged
Aged
Total
Healthcare
Aged
Male
Aged
Aged
Aged
Female
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Healthcare
Aged
Female
Unknown
Aged
Aged
Total
Aged
Male
Unknown
Aged
Aged
Female
Aged
Aged
Aged
Aged
Aged
Healthcare
Aged
Aged
Total
Male
Aged
Aged
Aged
Female
Aged
Aged
Unknown
Male
Aged
Aged
Total
Aged
Healthcare
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Total
Healthcare
Aged
Female
Unknown
Aged
Male
Aged
Aged
Aged
Healthcare
Aged
Female
Aged
Aged
Aged
Aged
Aged
Total
Unknown
Aged
Aged
Aged
Aged
Male
Aged
Unknown
Male
Aged
Aged
Aged
Total
Aged
Aged
Aged
Healthcare
Aged
Female
Aged
Aged
Aged
Aged
Male
Total
Female
Unknown
Aged
Healthcare
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Total
Aged
Aged
Aged
Aged
Unknown
Aged
Aged
Aged
Female
Aged
Aged
Healthcare
Male
Aged
Aged
Aged
Aged
Aged
Aged
Healthca

Unknown
Male
Aged
Aged
Healthcare
Aged
Unknown
Aged
Aged
Aged
Male
Aged
Aged
Aged
Aged
Female
Aged
Total
Aged
Aged
Aged
Aged
Aged
Male
Aged
Aged
Aged
Total
Healthcare
Female
Aged
Aged
Unknown
Aged
Aged
Aged
Aged
Aged
Male
Unknown
Aged
Aged
Total
Female
Healthcare
Aged
Aged
Aged
Total
Aged
Aged
Aged
Aged
Aged
Aged
Unknown
Female
Aged
Aged
Aged
Healthcare
Aged
Male
Male
Female
Total
Aged
Aged
Aged
Aged
Aged
Aged
Healthcare
Unknown
Aged
Aged
Aged
Aged
Total
Aged
Male
Aged
Female
Aged
Healthcare
Aged
Aged
Aged
Aged
Aged
Unknown
Aged
Aged
Aged
Aged
Aged
Female
Aged
Aged
Aged
Total
Aged
Male
Unknown
Aged
Healthcare
Aged
Aged
Aged
Female
Aged
Aged
Aged
Male
Unknown
Aged
Aged
Healthcare
Total
Aged
Aged
Aged
Aged
Unknown
Female
Aged
Aged
Total
Aged
Healthcare
Aged
Aged
Aged
Aged
Aged
Aged
Male
Aged
Aged
Aged
Female
Aged
Aged
Male
Aged
Aged
Unknown
Aged
Aged
Total
Healthcare
Aged
Aged
Aged
Female
Aged
Unknown
Aged
Aged
Healthcare
Total
Male
Aged
Aged
Aged
Aged
Aged
Aged
Aged
Unknown
Aged
Aged
Ag

In [30]:
column_list = pd.Series(column_list)
column_list = column_list.reset_index()

In [31]:
df1 = pd.merge(df1, column_list, left_index = True, right_index = True)

In [32]:
df1 = df1.rename(columns={'Column': "old_column", 0 : "Column"})

In [33]:
df1 = df1[[
    "Date",
    "old_column",
    "Column",
    "Age_Range",
    "Cases"
]]

In [34]:
df1.head()

,Date,old_column,Column,Age_Range,Cases
0,2020-02-29,ConfirmedCovidCases,Confirmed Covid Cases,NaN,1
1,2020-02-29,TotalConfirmedCovidCases,Total,NaN,1
2,2020-02-29,ConfirmedCovidDeaths,Confirmed Covid Deaths,NaN,0
3,2020-02-29,TotalCovidDeaths,Total Covid Deaths,NaN,0
4,2020-03-03,TotalCovidDeaths,Total Covid Deaths,NaN,0


### Daily Cases

In [35]:
hospitalised_df = df1[df1['Column'].str.contains("Hospitalised Aged")]
age_range_65up = hospitalised_df[hospitalised_df['Age_Range'] == '65-74']
age_range_65up.tail(10)

,Date,old_column,Column,Age_Range,Cases
20178,2021-11-29,HospitalisedAged65to74,Hospitalised Aged,65-74,3519
20200,2021-11-30,HospitalisedAged65to74,Hospitalised Aged,65-74,3521
20249,2021-12-01,HospitalisedAged65to74,Hospitalised Aged,65-74,3528
20290,2021-12-02,HospitalisedAged65to74,Hospitalised Aged,65-74,3532
20300,2021-12-03,HospitalisedAged65to74,Hospitalised Aged,65-74,3540
20336,2021-12-04,HospitalisedAged65to74,Hospitalised Aged,65-74,3550
20358,2021-12-05,HospitalisedAged65to74,Hospitalised Aged,65-74,3558
20402,2021-12-06,HospitalisedAged65to74,Hospitalised Aged,65-74,3559
20444,2021-12-07,HospitalisedAged65to74,Hospitalised Aged,65-74,3560
20475,2021-12-08,HospitalisedAged65to74,Hospitalised Aged,65-74,3567


In [36]:
age_list = df1['Age_Range'].unique()

# removing NaN from the dataset
age_list = age_list[1:]
print(age_list)

['75-84' '15-24' '35-44' '25-34' '5-14' '45-54' '55-64' '65-74']


In [37]:
aged_df = df1[df1['Column'].str.startswith("Aged")]

# dropping all NaN in hospitalised_df 
hospitalised_df = hospitalised_df[~hospitalised_df['Age_Range'].isna()]

# dropping all NaN in aged_df
aged_df = aged_df[~aged_df['Age_Range'].isna()]

# dropping all NaN in df1
new_df1 = df1[~df1['Column'].isna()]

In [38]:
aged_df.head()

,Date,old_column,Column,Age_Range,Cases
11,2020-03-04,Aged35to44,Aged,35-44,0
15,2020-03-04,Aged5to14,Aged,5-14,1
16,2020-03-04,Aged15to24,Aged,15-24,0
17,2020-03-04,Aged25to34,Aged,25-34,0
19,2020-03-04,Aged55to64,Aged,55-64,0


### Columns that are Not Accumulated
'Confirmed Covid Cases', 'Total Confirmed Covid Cases','Confirmed Covid Deaths', 'Total Covid Deaths',

In [39]:
df1['Column'].unique()

array(['Confirmed Covid Cases', 'Total', 'Confirmed Covid Deaths',
       'Total Covid Deaths', 'Hospitalised Aged', 'Aged',
       'Requiring Covid Cases', 'Median Age', 'Female', 'Unknown',
       'Hospitalised Covid Cases', 'Male', 'Healthcare',
       'Clusters Notified'], dtype=object)

In [40]:
column_list = df1['Column'].unique()
column_list

array(['Confirmed Covid Cases', 'Total', 'Confirmed Covid Deaths',
       'Total Covid Deaths', 'Hospitalised Aged', 'Aged',
       'Requiring Covid Cases', 'Median Age', 'Female', 'Unknown',
       'Hospitalised Covid Cases', 'Male', 'Healthcare',
       'Clusters Notified'], dtype=object)

In [41]:
# Columns that have accumulated cases and no daily cases count 
accumulated = ['Male','Hospitalised Covid Cases', 'Hospitalised Aged', 
               'Healthcare Workers Covid Cases', 'Aged', 'Unknown', 'Female',
               'Median Age', 'Requiring Covid Cases', 'Clusters Notified']

In [42]:
df1_list = []

for i in column_list:
    if "Hospitalised Aged" == i: 
        for age in age_list:
            df1 = hospitalised_df[hospitalised_df['Age_Range'].str.contains(age)]
            df1 = pd.DataFrame(df1)
            df1['Daily Cases'] = df1['Cases'].diff(1)
            df1 = df1.dropna()
            df1["Daily Cases"] = df1["Daily Cases"].astype(int)
            df1_list.append(df1)
        
    elif "Aged" == i: 
        for age in age_list:
            df1 = aged_df[aged_df['Age_Range'].str.contains(age)]
            df1 = pd.DataFrame(df1)
            df1['Daily Cases'] = df1['Cases'].diff(1)
            df1 = df1.dropna()
            df1["Daily Cases"] = df1["Daily Cases"].astype(int)
            df1_list.append(df1)
    
    elif  i in accumulated:
        df1 = new_df1[new_df1['Column'].str.contains(i)]
        df1 = pd.DataFrame(df1)
        df1['Daily Cases'] = df1['Cases'].diff(1)
#         df1 = df1.dropna()
#         df1["Daily Cases"] = df1["Daily Cases"].astype(int)
        df1_list.append(df1)
    else:
        df1 = df1[df1['Column'].str.contains(i)]
        df1 = pd.DataFrame(df1)
        df1['Daily Cases'] = 'NaN'
#         df1 = df1.dropna()
#         df1["Daily Cases"] = df1["Daily Cases"].astype(int)
        df1_list.append(df1) 

In [43]:
df1 =  pd.concat(df1_list)

In [44]:
# sorting data by date
df1 = df1.sort_values(ascending = True, by = ["Date"])

# reseting the dataframe index
df1 = df1.reset_index(drop=True)

In [45]:
df1.head(50)

,Date,old_column,Column,Age_Range,Cases,Daily Cases
0,2020-02-29,ConfirmedCovidCases,Confirmed Covid Cases,NaN,1,NaN
1,2020-03-03,ConfirmedCovidCases,Confirmed Covid Cases,NaN,1,NaN
2,2020-03-04,Female,Female,NaN,0,NaN
3,2020-03-04,HospitalisedCovidCases,Hospitalised Covid Cases,NaN,0,NaN
4,2020-03-04,Median_Age,Median Age,NaN,13,NaN
5,2020-03-04,Male,Male,NaN,1,NaN
6,2020-03-04,Unknown,Unknown,NaN,0,NaN
7,2020-03-04,ConfirmedCovidCases,Confirmed Covid Cases,NaN,4,NaN
8,2020-03-04,RequiringICUCovidCases,Requiring Covid Cases,NaN,0,NaN
9,2020-03-05,HospitalisedAged25to34,Hospitalised Aged,25-34,0,0


In [46]:
df1.tail(50)

,Date,old_column,Column,Age_Range,Cases,Daily Cases
15396,2021-12-06,Unknown,Unknown,NaN,290,8.0
15397,2021-12-06,Aged5to14,Aged,5-14,80027,1137
15398,2021-12-07,Male,Male,NaN,292327,1456.0
15399,2021-12-07,HospitalisedAged75to84,Hospitalised Aged,75-84,4126,1
15400,2021-12-07,ConfirmedCovidCases,Confirmed Covid Cases,NaN,5590,NaN
15401,2021-12-07,HospitalisedAged15to24,Hospitalised Aged,15-24,1037,1
15402,2021-12-07,HospitalisedAged35to44,Hospitalised Aged,35-44,1912,3
15403,2021-12-07,ClustersNotified,Clusters Notified,NaN,0,0.0
15404,2021-12-07,HospitalisedAged25to34,Hospitalised Aged,25-34,1607,1
15405,2021-12-07,Aged5to14,Aged,5-14,80671,644


In [47]:
check = df1[df1['Column'].str.startswith("Aged")]
age_range = check[check['Age_Range'] == '5-14']
age_range.tail(10)

,Date,old_column,Column,Age_Range,Cases,Daily Cases
15223,2021-11-29,Aged5to14,Aged,5-14,71935,842
15235,2021-11-30,Aged5to14,Aged,5-14,73041,1106
15254,2021-12-01,Aged5to14,Aged,5-14,74260,1219
15298,2021-12-02,Aged5to14,Aged,5-14,75158,898
15307,2021-12-03,Aged5to14,Aged,5-14,76123,965
15343,2021-12-04,Aged5to14,Aged,5-14,77556,1433
15368,2021-12-05,Aged5to14,Aged,5-14,78890,1334
15397,2021-12-06,Aged5to14,Aged,5-14,80027,1137
15405,2021-12-07,Aged5to14,Aged,5-14,80671,644
15431,2021-12-08,Aged5to14,Aged,5-14,81898,1227


### Saving to a new csv file

In [48]:
df1.to_csv(r"tidy_data_df.csv", index =  False)